# Train/fine-tune computer vision (CV) classifiers

In this notebook, we use the annotated images (see, e.g., notebooks `001` and `002`) to train/fine-tune CV classifiers.

In [ ]:
# solve issue with autocomplete
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from mapreader import classifier
from mapreader import loadAnnotations
from mapreader import patchTorchDataset

import numpy as np
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torchvision import models

## Read annotations

In [ ]:
annotated_images = loadAnnotations()

In [ ]:
annotated_images.load("./annotations_phenotype_open_access/phenotype_test_#kasra#.csv", 
                      path2dir="./dataset/eg_slice_50_50")

In [ ]:
annotated_images.annotations.columns.tolist()

In [ ]:
print(annotated_images)

In [ ]:
# We need to shift these labels so that they start from 0:
annotated_images.adjust_labels(shiftby=-1)

In [ ]:
# show sample images for target label (tar_label)
annotated_images.show_image_labels(tar_label=1, num_sample=6)

In [ ]:
# show an image based on its index 
annotated_images.show_image(indx=2)

### Split annotations into train/val or train/val/test

We use a stratified method for splitting the annotations, that is, each set contains approximately the same percentage of samples of each target label as the original set.

In [ ]:
annotated_images.split_annotations(frac_train=0.8, 
                                   frac_val=0.2, 
                                   frac_test=0.0)

Dataframes for train, validation and test sets can be accessed via:

```python
annotated_images.train
annotated_images.val
annotated_images.test
```

In [ ]:
annotated_images.train["label"].value_counts()

In [ ]:
annotated_images.val["label"].value_counts()

In [ ]:
# annotated_images.test["label"].value_counts()

# Classifier

## Dataset

Define transformations to be applied to images before being used in training or validation/inference.

`patchTorchDataset` has some default transformations. However, it is possible to define your own transformations and pass them to `patchTorchDataset`:

In [ ]:
# ------------------
# --- Transformation
# ------------------ 

# FOR INCEPTION
#resize2 = 299
# otherwise:
resize2 = 224

# mean and standard deviations of pixel intensities in 
# all the patches in 6", second edition maps
normalize_mean = 1 - np.array([0.82860442, 0.82515008, 0.77019864])
normalize_std = 1 - np.array([0.1025585, 0.10527616, 0.10039222])
# other options:
# normalize_mean = [0.485, 0.456, 0.406]
# normalize_std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose(
        [transforms.Resize(resize2),
         transforms.RandomApply([
             transforms.RandomHorizontalFlip(p=0.5),
             transforms.RandomVerticalFlip(p=0.5),
             ], p=0.5),
#         transforms.RandomApply([
#             transforms.GaussianBlur(21, sigma=(0.5, 5.0)),
#             ], p=0.25),
         transforms.RandomApply([
             #transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
             transforms.Resize((50, 50)),
             ], p=0.25),
#          transforms.RandomApply([
#              transforms.RandomAffine(180, translate=None, scale=None, shear=20),
#              ], p=0.25),
        transforms.Resize((resize2, resize2)),
        transforms.ToTensor(),
        transforms.Normalize(normalize_mean, normalize_std)
        ]),
    'val': transforms.Compose(
        [transforms.Resize((resize2, resize2)),
        transforms.ToTensor(),
        transforms.Normalize(normalize_mean, normalize_std)
        ]),
}


Now, we can use these transformations to instantiate `patchTorchDataset`:

In [ ]:
train_dataset = patchTorchDataset(annotated_images.train, 
                                  transform=data_transforms["train"])
valid_dataset = patchTorchDataset(annotated_images.val,   
                                  transform=data_transforms["val"])
# test_dataset  = patchTorchDataset(annotated_images.test,  
#                                   transform=data_transforms["val"])

## Sampler

In [ ]:
# -----------
# --- Sampler
# -----------
# We define a sampler as we have a highly imbalanced dataset
label_counts_dict = annotated_images.train["label"].value_counts().to_dict()

class_sample_count = []
for i in range(0, len(label_counts_dict)):
    class_sample_count.append(label_counts_dict[i])
    
weights = 1. / (torch.Tensor(class_sample_count)/1.)
weights = weights.double()
print(f"Weights: {weights}")

train_sampler = torch.utils.data.sampler.WeightedRandomSampler(
    weights[train_dataset.patchframe["label"].to_list()], 
    num_samples=len(train_dataset.patchframe))

valid_sampler = torch.utils.data.sampler.WeightedRandomSampler(
    weights[valid_dataset.patchframe["label"].to_list()], 
    num_samples=len(valid_dataset.patchframe))


## Dataloader

In [ ]:
myclassifier = classifier(device="default")
# myclassifier.load("./checkpoint_12.pkl")

In [ ]:
batch_size = 8

# Add training dataset
myclassifier.add2dataloader(train_dataset, 
                            set_name="train", 
                            batch_size=batch_size, 
                            # shuffle can be False as annotations have already been shuffled
                            shuffle=False,
                            num_workers=0,
                            sampler=train_sampler
                           )

# Add validation dataset
myclassifier.add2dataloader(valid_dataset, 
                            set_name="val", 
                            batch_size=batch_size, 
                            shuffle=False, 
                            num_workers=0,
                            #sampler=valid_sampler
                           )

In [ ]:
myclassifier.print_classes_dl()

In [ ]:
# set class names for plots
class_names = {0: "No", 
               1: "Plant"}
myclassifier.set_classnames(class_names)

In [ ]:
myclassifier.print_classes_dl()

In [ ]:
myclassifier.batch_info()

In [ ]:
for bn in range(1, 3):
    myclassifier.show_sample(set_name="train", 
                             batch_number=bn, 
                             print_batch_info=False)

## Load a (pretrained) PyTorch model and add it to `classifier`

Two methods to add a (pretrained) PyTorch model:
1. Define a model using `from torchvision import models`
2. Use `.initialize_model` method

### Method 1: Define a model using `from torchvision import models`

In [ ]:
# # Choose a model from the supported PyTorch models
# model_ft = models.resnet18(pretrained=True)

# # Add FC based on the number of classes
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, myclassifier.num_classes)

# # Add the model to myclassifier
# myclassifier.add_model(model_ft)

In [ ]:
# myclassifier.model_summary()

### Method 2: use `.initialize_model`

In [ ]:
myclassifier.del_model()

In [ ]:
myclassifier.initialize_model("resnet18", 
                              pretrained=True, 
                              last_layer_num_classes="default",
                              add_model=True)

In [ ]:
myclassifier.model_summary(only_trainable=False)

## (Un)freeze layers in the neural network architecture

In [ ]:
# myclassifier.freeze_layers(["conv1.weight", "bn1.weight", "bn1.bias", "layer1*", "layer2*", "layer3*"])
# myclassifier.model_summary(only_trainable=False)

In [ ]:
# myclassifier.unfreeze_layers(["layer3*"])
# myclassifier.model_summary(only_trainable=False)

In [ ]:
# myclassifier.only_keep_layers(["fc.weight", "fc.bias"])
# myclassifier.model_summary(only_trainable=True)

## Define optimizer, scheduler and criterion

We can either use one learning rate for all the layers in the neural network or define layerwise learning rates, that is, the learning rate of each layer is different. This is normally used in fine-tuning pretrained models in which a smaller learning rate is assigned to the first layers.

`MapReader` has a `.layerwise_lr` method to define layerwise learning rates. By default, `MapReader` uses a linear function to distribute the learning rates (using `min_lr` for the first layer and `max_lr` for the last layer). The linear function can be changed using `ltype="geomspace"` argument.

In [ ]:
list2optim = myclassifier.layerwise_lr(min_lr=1e-4, max_lr=1e-3)
# #list2optim = myclassifier.layerwise_lr(min_lr=1e-4, max_lr=1e-3, ltype="geomspace")

In [ ]:
optim_param_dict = {
    "lr": 1e-3,
    "betas": (0.9, 0.999), 
    "eps": 1e-08, 
    "weight_decay": 0, 
    "amsgrad": False
}

# --- if list2optim is defined, e.g., by using `.layerwise_lr` method (see the previous cell):
myclassifier.initialize_optimizer(optim_type="adam", 
                                  params2optim=list2optim,
                                  optim_param_dict=optim_param_dict,
                                  add_optim=True)

# --- otherwise:
# myclassifier.initialize_optimizer(optim_type="adam", 
#                                   optim_param_dict=optim_param_dict,
#                                   add_optim=True)

Other optimizers can also be used in the above cell, e.g.:

```python
optim_param_dict = {
    "lr": 1e-3, 
    "momentum": 0, 
    "dampening": 0, 
    "weight_decay": 0, 
    "nesterov": False
}

myclassifier.initialize_optimizer(optim_type="sgd", 
                                  optim_param_dict=optim_param_dict,
                                  add_optim=True)
```

In [ ]:
scheduler_param_dict = {
    "step_size": 10, 
    "gamma": 0.1, 
    "last_epoch": -1, 
#    "verbose": False
}

myclassifier.initialize_scheduler(scheduler_type="steplr",
                                  scheduler_param_dict=scheduler_param_dict,
                                  add_scheduler=True)

Other schedulers can also be used in the above cell, e.g.:

```python
scheduler_param_dict = {
    "max_lr": 1e-2, 
    "steps_per_epoch": len(myclassifier.dataloader["train"]), 
    "epochs": 5
}

myclassifier.initialize_scheduler(scheduler_type="OneCycleLR",
                                  scheduler_param_dict=scheduler_param_dict,
                                  add_scheduler=True)
```

In [ ]:
# Add criterion
criterion = nn.CrossEntropyLoss()

myclassifier.add_criterion(criterion)

## Train/fine-tune a model

In [ ]:
myclassifier.train_component_summary()

**Note:** it is possible to interrupt a training (using Kernel/Interrupt in Jupyter Notebook or ctrl+C). 

In [ ]:
myclassifier.train(num_epochs=10, 
                   save_model_dir="./models_plant_open", 
                   tensorboard_path="tboard_plant_open", 
                   verbosity_level=0,
                   tmp_file_save_freq=2,
                   remove_after_load=False,
                   print_info_batch_freq=5)

### Plot results

In [ ]:
list(myclassifier.metrics.keys())

In [ ]:
myclassifier.plot_metric(y_axis=["epoch_loss_train", "epoch_loss_val"],
                         y_label="Loss",
                         legends=["Train", "Valid"],
                         colors=["k", "tab:red"])

In [ ]:
myclassifier.plot_metric(y_axis=["epoch_rocauc_macro_train", "epoch_rocauc_macro_val"],
                         y_label="ROC AUC",
                         legends=["Train", "Valid"],
                         colors=["k", "tab:red"])

In [ ]:
myclassifier.plot_metric(y_axis=["epoch_fscore_macro_train", 
                                 "epoch_fscore_macro_val", 
                                 "epoch_fscore_0_val", 
                                 "epoch_fscore_1_val"],
                         y_label="F-score",
                         legends=["Train", 
                                  "Valid", 
                                  "Valid (label: 0)",
                                  "Valid (label: 1)",],
                         colors=["k", "tab:red", "tab:red", "tab:red"],
                         styles=["-", "-", "--", ":"],
                         markers=["o", "o", "", ""],
                         plt_yrange=[0, 100])

In [ ]:
myclassifier.plot_metric(y_axis=["epoch_recall_macro_train", 
                                 "epoch_recall_macro_val", 
                                 "epoch_recall_0_val", 
                                 "epoch_recall_1_val"],
                         y_label="Recall",
                         legends=["Train", 
                                  "Valid", 
                                  "Valid (label: 0)",
                                  "Valid (label: 1)",],
                         colors=["k", "tab:red", "tab:red", "tab:red"],
                         styles=["-", "-", "--", ":"],
                         markers=["o", "o", "", ""],
                         plt_yrange=[0, 100])